## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Latitude,longitude,Maximum temperature,Percent humidity,Percent cloudiness,Wind Speed,Weather description
0,0,Mutsu,JP,41.2894,141.2169,61.12,85,100,11.12,overcast clouds
1,1,Mataura,NZ,-46.1927,168.8643,47.48,98,100,5.75,light rain
2,2,Aklavik,CA,68.2191,-135.0107,30.92,79,100,6.31,overcast clouds
3,3,Kruisfontein,ZA,-34.0033,24.7314,48.88,74,37,6.04,scattered clouds
4,4,Hay River,CA,60.8156,-115.7999,39.31,70,91,14.34,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Maximum temperature"] <= max_temp) & \
                                       (city_data_df["Maximum temperature"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Latitude,longitude,Maximum temperature,Percent humidity,Percent cloudiness,Wind Speed,Weather description
6,6,Atuona,PF,-9.8000,-139.0333,78.39,76,27,19.48,scattered clouds
12,12,Faanui,PF,-16.4833,-151.7500,79.79,74,44,12.84,scattered clouds
15,15,Hithadhoo,MV,-0.6000,73.0833,82.72,68,99,15.14,overcast clouds
16,16,Hilo,US,19.7297,-155.0900,81.39,79,75,10.36,moderate rain
17,17,Vaini,TO,-21.2000,-175.2000,77.16,83,75,9.22,broken clouds
22,22,Nileshwar,IN,12.2500,75.1000,77.54,90,35,5.35,scattered clouds
27,27,Marshall,US,34.3834,-86.3333,75.61,73,1,1.01,clear sky
29,29,Kapaa,US,22.0752,-159.3190,84.18,71,20,5.01,light rain
35,35,Songkhla,TH,6.8333,100.6667,77.05,88,26,3.65,scattered clouds
51,51,Meulaboh,ID,4.1363,96.1285,76.17,86,100,3.00,overcast clouds


In [15]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                210
City                   210
Country                210
Latitude               210
longitude              210
Maximum temperature    210
Percent humidity       210
Percent cloudiness     210
Wind Speed             210
Weather description    210
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=clean_df.dropna()

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Maximum temperature", "Weather description", "Latitude", "longitude"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Maximum temperature,Weather description,Latitude,longitude,Hotel Name
6,Atuona,PF,78.39,scattered clouds,-9.8000,-139.0333,
12,Faanui,PF,79.79,scattered clouds,-16.4833,-151.7500,
15,Hithadhoo,MV,82.72,overcast clouds,-0.6000,73.0833,
16,Hilo,US,81.39,moderate rain,19.7297,-155.0900,
17,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,
22,Nileshwar,IN,77.54,scattered clouds,12.2500,75.1000,
27,Marshall,US,75.61,clear sky,34.3834,-86.3333,
29,Kapaa,US,84.18,light rain,22.0752,-159.3190,
35,Songkhla,TH,77.05,scattered clouds,6.8333,100.6667,
51,Meulaboh,ID,76.17,overcast clouds,4.1363,96.1285,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Latitude"]
    lng = row["longitude"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df.dropna()
clean_hotel_df

,City,Country,Maximum temperature,Weather description,Latitude,longitude,Hotel Name
6,Atuona,PF,78.39,scattered clouds,-9.8000,-139.0333,Villa Enata
12,Faanui,PF,79.79,scattered clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
15,Hithadhoo,MV,82.72,overcast clouds,-0.6000,73.0833,Scoop Guest House
16,Hilo,US,81.39,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
17,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
650,Praia,CV,79.34,few clouds,14.9215,-23.5087,LT Aparthotel
655,Rach Gia,VN,82.35,overcast clouds,10.0167,105.0833,Nhà Hàng Hòa Giang
666,Umm Lajj,SA,86.74,clear sky,25.0213,37.2685,Moon light Furnished Units
667,Magaria,NE,83.10,overcast clouds,12.9983,8.9099,Chez Honorable Chapiou


In [32]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather description} and {Maximum temperature} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Latitude", "longitude"]]

In [41]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))